#### Assumptions
- It is possible that some variable is "free", i.e., its value can be taken any number in the field. I have not considered that as it makes symbolic root solving for other variables difficult.

In [1]:
set_random_seed(3)

In [2]:
n = 3 # Number of variables
m = 3 # Number of equations

In [3]:
F = RR # Field 

In [4]:
variables = ['x'+str(i) for i in range(n)]; variables

['x0', 'x1', 'x2']

In [5]:
P = PolynomialRing(F, variables, order='lex'); P

Multivariate Polynomial Ring in x0, x1, x2 over Real Field with 53 bits of precision

In [6]:
equations = [P.random_element() for i in range(m)]; equations

[0.926172588505559*x0*x1 + 0.584803390058547*x0*x2 - 0.247267743493293*x1^2 - 0.873387401036998*x1 - 0.655064707635064*x2^2,
 -0.492643731563807*x0*x1 + 0.245862240622783*x0 - 0.517240129827284*x1 + 0.642662376505148*x2^2 - 0.0810015184273003,
 -0.522249059762958*x0^2 - 0.589913953878200*x0 - 0.387225601852989*x1 - 0.429439065777491*x2^2 - 0.324240018816532]

In [7]:
def generate_groebner(F,P,equations,n,m):
    I = Ideal(equations)
    gb = I.groebner_basis()
    gb_matrix = [list(gb[i].dict().items()) for i in range(len(gb))]
    
    variables_present = []
    for polynomial in gb_matrix:
        variables = []
        for term in polynomial:
            for i, variable in enumerate(term[0]):
                if variable != 0 and i not in variables:
                    variables.append(i)
        variables_present.append(variables)

    return gb, gb_matrix, variables_present

In [8]:
def solve_polynomial_system(gb, gb_matrix, variables_present):
    all_values = []
    backtrack_multivariate([], gb, gb_matrix, variables_present, len(gb_matrix), 0, all_values)
    return all_values

In [9]:
def backtrack_multivariate(values, gb, gb_matrix, variables_present, current_index, previous_index, all_values):
    x = P.gens()
    if len(gb_matrix) == 1 and sum(gb_matrix[0][0][0]) == 0 and gb_matrix[0][0][1] == 1:
        print("No Solution to given constraints!! Returning")
        return
    i = len(values)
    if i > n-1:
        all_values.append(values)
        return
    
    previous_index = current_index
    current_index = current_index - 1
    to_eliminate = n-2-i
    while current_index>=0:
            temp_bool = true
            for j in variables_present[current_index]:
                temp_bool = (temp_bool and (j>to_eliminate))
            if temp_bool is False:
                current_index = min(current_index + 1, len(gb)-1)
                break
            else:
                current_index = current_index - 1
    current_index = max(0, current_index)
    
    if i > 0:    
        gb_new = []
        for j in range(len(gb)):
            gb_new.append(gb[j].subs({x[n-1-i]:values[0]}))
        gb_matrix_new = [list(gb_new[z].dict().items()) for z in range(len(gb_new))];
    else: 
        gb_new = gb
        gb_matrix_new = gb_matrix
    
    all_roots = []
    for k in range(current_index, previous_index):
        roots = []
        
        try:
            factors = gb_new[k].factor()
        except:
            continue
        for fact in factors:
            polynomial = fact[0].dict()
            fac = list(polynomial.items())
            if len(fac) == 1:
                if 0 not in roots:
                    roots.append(0)
                
            if len(fac) == 2:
                if sum(fac[1][0]) < 2:
                    value = -fac[0][1]/fac[1][1]
                    if value not in roots:
                        roots.append(value)
        all_roots.append(roots)
    
    if (len(all_roots)) == 0:
        print("Could not proceed with values ", values, "returning")
        print("Possible reason: No solution in the given field or a variable can take infinite values")
        return
    common_roots = all_roots[0]
    for j in range(1,len(all_roots)):
        common_roots = set(common_roots).intersection(all_roots[j])
    if len(common_roots) == 0:
        try:
            common_roots.add(0)
        except:
            try:
                common_roots.append(0)
            except:
                return
    
    for root in common_roots:
        new = values
        new.insert(0,root)
        backtrack_multivariate(new.copy(), gb_new.copy(), gb_matrix_new.copy(), variables_present, current_index, previous_index, all_values)
        values = values[1:]
        

In [10]:
def print_solutions(solutions):
    for sol in solutions:
        print(sol)

In [11]:
gb, gb_matrix, variables_present = generate_groebner(F,P,equations,n,m)

In [12]:
solutions = solve_polynomial_system(gb, gb_matrix, variables_present)

In [13]:
print_solutions(solutions)

[-2.71562532770562, 2.54213091672306, 2.10530008186475]
[-2.71180598484918, 2.41680896343768, -2.54236010447131]
[-2.67206102923306, 1.19862029213984, -2.54236010447131]
[2.30258044980207, -2.15908558181135, -2.54236010447131]
[1.81364943740727, -2.15908558181135, -2.54236010447131]
[1.06652197074627, -2.15908558181135, -2.54236010447131]
[-1.15477208901541, -2.15908558181135, -2.54236010447131]
[-2.52302453202815, -2.15908558181135, -2.54236010447131]
